# On trigger signal acquisition

## Description

This example shows how to acquire 16k samples of signal on fast analog inputs.
Signal will be acquired when the internal trigger condition is meet.
Time length of the acquired signal depends on the time scale of a buffer
that can be set with a decimation factor.

TODO: describe some calculations

## Required hardware

- Red Pitaya
- Signal (function) generator

![wiring diagram](img/on_given_trigger_acquire_signal_on_fast_analog_input.png)

The `rp` object is an instance of the `redpitaya` class.
When the object is initialized, the FPGA bitstream is loaded and
memory mapped FPGA registers are mapped into software.
Repeating FPGA bitstream loading will cause all registers to reset,
while mapping the memory space multiple times will cause segmentation faults.
So untill this issues are handled by the driver
a `redpitaya` instance should be created only once.

In [ ]:
from mercury import overlay
o = overlay("mercury")

In [ ]:
# generator configuration
import numpy as np
from mercury import gen

# instantiate generator 0 (the other is unused)
gen0 = gen(0)

# create a sinusoid curve
x = np.linspace(-np.pi, np.pi, gen0.N, endpoint=False)
wave = np.sin(x)

gen0.amplitude = 1.0
gen0.offset    = 0
# waveform should be loaded before frequency and phase are calculated
# this provides the correct buffer size
gen0.waveform  = wave
gen0.frequency = 10000
gen0.phase     = 0
gen0.mask      = [0x1, 0x2, 0x4]
gen0.start()
gen0.trigger()

#del(gen0)

In [ ]:
# acquisition configuration
from mercury import osc

size = 1024
channels = (0, 1)
oscs = [osc(ch, 1.0) for ch in channels]

for ch in channels:
    # TODO: for now bypass input filter
    oscs[ch].filter_bypass = True

    # trigger timing
    oscs[ch].trigger_pre_delay  = size//4*1
    oscs[ch].trigger_post_delay = size//4*4

    # trigger level [V], edge and holdoff [periods]
    oscs[ch].level = [-0.1, +0.1]
    oscs[ch].edg = 'pos'
    oscs[ch].holdoff = 20

    # trigger source mask
    mask = [0x1<<10, 0x2<<10, 0x4<<10]
    mask[2] |= 0x8<<10
    oscs[ch].mask = mask

    oscs[ch].reset()

oscs[0].start()
#oscs[0].trigger()
while oscs[0].status_run(): pass
print('triggered')

# read data from FPGA FIFO into memory and display it
buff = [oscs[ch].data(size) for ch in channels];

In [ ]:
import time
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool, Range1d
from bokeh.plotting import figure
from bokeh.resources import INLINE 
output_notebook(resources=INLINE)

In [ ]:
x = np.arange(size) / rp.FS

colors = ('red', 'blue')
hover = HoverTool(mode = 'vline', tooltips=[("T", "@x"), ("V", "@y")])
tools = "pan,wheel_zoom,box_zoom,reset,crosshair"
p = figure(plot_height=500, plot_width=900, title="oscilloscope", toolbar_location="above", tools=(tools, hover))
p.xaxis.axis_label='time [s]'
p.yaxis.axis_label='voltage [V]'
p.y_range=Range1d(-1.2, 1.2)
r = [p.line(x, buff[i], line_width=1, line_alpha=0.7, color=colors[i]) for i in channels]

# get an explicit handle to update the next show cell with
target = show(p, notebook_handle=True)

In [ ]:
while True:
    oscs[0].reset()
    oscs[0].start()
    while oscs[0].status_run(): pass
    buff = [oscs[ch].data(size) for ch in channels];
#    buff = np.absolute(np.fft.fft(buff))
    for i in channels:
        r[i].data_source.data['y'] = buff[i]
    # push updates to the plot continuously using the handle (intererrupt the notebook kernel to stop)
    push_notebook(handle=target)
#    time.sleep(0.05)